In [38]:
import numpy as np
import os
import random
import pickle

def read_pgm(fileName):
    with open(fileName, 'rb') as f:
        header = f.readline()
        width, height = map(int, f.readline().split())
        maxVal = int(f.readline().strip())
        imageData = []
        for _ in range(height):
            row = []
            for _ in range(width):
                row.append(int.from_bytes(f.read(1), 'big'))
            imageData.append(row)
    return np.array(imageData)

def generate_binary_patches_from_pgm(pgm_folder, num_patches_per_file=1000, patch_size=2):
    binary_patches = []
    
    for file_name in os.listdir(pgm_folder):
        if file_name.endswith(".pgm"):
            pgm_file = os.path.join(pgm_folder, file_name)
            imageData = read_pgm(pgm_file)
            height, width = imageData.shape
            
            for _ in range(num_patches_per_file):
                # Select a random patch
                x = random.randint(0, width - patch_size)
                y = random.randint(0, height - patch_size)
                patch = imageData[y:y+patch_size, x:x+patch_size]
                
                # Convert patch to unsigned byte data type
                patch = patch.astype(np.uint8)

                # Convert patch to binary vectors for individual pixels
                binary_patch = np.unpackbits(patch).reshape(-1, 8)

                # Add binary patch to list
                binary_patches.append(binary_patch)

                # Print the generated patch
                print(f"Patch {len(binary_patches)}:", [list(pixel) for pixel in binary_patch])
    
    return binary_patches

def imprint_patches(weightMatrix, patches):
    n = len(weightMatrix)
    num_pixels = len(patches[0])  # Number of pixels in each patch
    for patch in patches:
        for i in range(num_pixels):
            for j in range(num_pixels):
                if i != j:
                    for k in range(8):  # Iterate over each bit in the pixel
                        weightMatrix[i * 8 + k, j * 8 + k] += (2 / n) * (patch[i][k] * 2 - 1) * (patch[j][k] * 2 - 1)
    return weightMatrix

def save_network(network, fileName):
    with open(fileName, 'wb') as file:
        pickle.dump(network, file)

# Example usage
if __name__ == "__main__":
    # Step 1: Generate Binary Patches
    pgm_folder_with_patches = "pgm images"
    binary_patches = generate_binary_patches_from_pgm(pgm_folder_with_patches)
    
    # Step 2: Define Number of Neurons
    num_neurons = len(binary_patches[0]) * 8  # Total number of neurons
    
    # Step 3: Train Hopfield Network
    initial_weightMatrix = np.zeros((num_neurons, num_neurons))
    weightMatrix = imprint_patches(initial_weightMatrix, binary_patches)
    
    # Step 4: Save Hopfield Network
    save_network(weightMatrix, "hopfield_network_2.pkl")



Patch 1: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 2: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 3: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 4: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 5: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 6: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 7: [[1, 1, 0, 1, 1, 1, 0, 0], [1, 1, 0, 1, 0, 0, 1, 1], [1, 1, 1, 0, 0, 0, 1, 1], [1, 1, 1, 0, 0, 1, 0, 1]]
Patch 8: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1]]
Patch 9: [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1], 

In [46]:
import numpy as np
import os
import random
import pickle

def load_network(fileName):
    with open(fileName, 'rb') as file:
        network = pickle.load(file)
    return np.array(network)  # Convert to numpy array to ensure correct shape

def pgm_to_binary_vectors(fileName):
    width, height, max_val, imageData = read_pgm(fileName)

    binaryVectors = []
    for row in imageData:
        for value in row:
            binaryVector = np.array([int(bit) for bit in format(value, '08b')])
            binaryVectors.append(binaryVector)

    # Split binary vectors into patches of 4 pixels
    patches = [binaryVectors[i:i+4] for i in range(0, len(binaryVectors), 4)]

    return patches

def save_pgm_from_binary_vectors(binaryVectors, fileName):
    num_patches = len(binaryVectors)
    patch_size = binaryVectors[0].shape[0]
    pixels_per_patch = binaryVectors[0].shape[1]

    # Set image dimensions to 256x256
    width = 128
    height = 128

    # Convert binary vectors back to pixel values
    pixelValues = [int(''.join(map(str, pixel.flatten())), 2) for patch in binaryVectors for pixel in patch]

    # Reshape pixel values into image array
    imageArray = np.array(pixelValues).reshape(height, width)

    # Save the image
    write_pgm(fileName, width, height, 255, imageArray.tolist())

def predict(weightMatrix, patternArray, max_iterations=100):
    n = len(weightMatrix)
    predicted_patterns = []
    
    for input_pattern in patternArray:
        for _ in range(max_iterations):
            old_pattern = input_pattern.copy()
            for i in range(n):
                hi = np.dot(weightMatrix[i], input_pattern)
                input_pattern[i] = 0 if hi < 0 else 1  # Change thresholding here
            if np.array_equal(input_pattern, old_pattern):
                break
        predicted_patterns.append(input_pattern)
    return np.array(predicted_patterns)

def read_pgm(fileName):
    with open(fileName, 'rb') as f:
        header = f.readline()
        width, height = map(int, f.readline().split())
        maxVal = int(f.readline().strip())
        imageData = []
        for _ in range(height):
            row = []
            for _ in range(width):
                row.append(int.from_bytes(f.read(1), 'big'))
            imageData.append(row)
    return width, height, maxVal, imageData

def write_pgm(fileName, width, height, maxVal, imageData):
    with open(fileName, 'wb') as f:
        f.write(b'P5\n')
        f.write(f"{width} {height}\n".encode())
        f.write(f"{maxVal}\n".encode())
        for row in imageData:
            for pixel in row:
                f.write(bytes([pixel]))

if __name__ == "__main__":

    hopfield_network = load_network("hopfield_network_2.pkl")

    pgm_folder_for_prediction = "128x128 compressed pgm images"
    for file_name in os.listdir(pgm_folder_for_prediction):
        if file_name.endswith(".pgm"):
            pgm_file = os.path.join(pgm_folder_for_prediction, file_name)
            patches = pgm_to_binary_vectors(pgm_file)

            predicted_patches = []
            for patch in patches:
                flattened_patch = np.concatenate(patch)
                predicted_patch = predict(hopfield_network, [flattened_patch])[0]
                predicted_patches.append(predicted_patch.reshape(-1, 8))

            predicted_pgm_file = os.path.join("predicted images", file_name)
            save_pgm_from_binary_vectors(predicted_patches, predicted_pgm_file)


KeyboardInterrupt: 